# Compare genres of randomly generated recommendations with original books

In [1]:
import gensim
from gensim.models import Doc2Vec, KeyedVectors
import logging
import time
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import requests
import matplotlib.pyplot as plt
import gensim.downloader as api # Downloading the Google pretrained Word2Vec Model

In [2]:
import numpy as np
import requests
import random
from tqdm.notebook import tqdm
tqdm.pandas()

In [3]:
import os
import json
import glob
import re
import sys
import collections
from nltk import flatten
import dask
from dask import delayed,compute
import dask.dataframe as dd
from dask.multiprocessing import get
import logging
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
from sklearn.metrics import multilabel_confusion_matrix
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from itertools import repeat
import random

In [5]:
#read dataset
books_df = pd.read_csv('unique_books.csv')
#drop na (subset-genre)
books_df = books_df.dropna(subset=['genres'])
#remove quotes from around genre list
books_df['genres'] = books_df.loc[books_df['genres']!=np.nan, 'genres'].progress_apply(lambda x: ast.literal_eval(x))

  0%|          | 0/4326 [00:00<?, ?it/s]

In [7]:
books_df.shape

(4326, 18)

In [6]:
#create list of genres
genres_list = []
for i in books_df['genres']:
    genres_list.extend(i)
#create unique list of genres
unique_genres_list = list(set(genres_list))
unique_genres_df = pd.DataFrame.from_dict({'genres': sorted(unique_genres_list)})
num_genres = len(unique_genres_list)
num_genres

424

In [9]:
orig_book_indices = list(books_df.index)

In [10]:
num_orig_books = len(orig_book_indices)
num_orig_books

4326

## Create random recommendations


In [19]:
#5 random recommendations for each book:
num_preds_per_book = 10
random_recs_df = pd.DataFrame()
for i in range(books_df.shape[0]): #for each book
    for j in range(num_preds_per_book):
        #pick random book from all books
        book_idx = random.choice(orig_book_indices)
        #add recommendation to the df
        random_recs_df = pd.concat([random_recs_df, books_df.iloc[books_df.index==book_idx]])
    print(i/(books_df.shape[0]))

0.0
0.0002311604253351826
0.0004623208506703652
0.0006934812760055479
0.0009246417013407304
0.001155802126675913
0.0013869625520110957
0.0016181229773462784
0.0018492834026814608
0.0020804438280166435
0.002311604253351826
0.002542764678687009
0.0027739251040221915
0.003005085529357374
0.003236245954692557
0.0034674063800277394
0.0036985668053629217
0.003929727230698105
0.004160887656033287
0.00439204808136847
0.004623208506703652
0.0048543689320388345
0.005085529357374018
0.0053166897827092
0.005547850208044383
0.005779010633379565
0.006010171058714748
0.0062413314840499305
0.006472491909385114
0.006703652334720296
0.006934812760055479
0.007165973185390661
0.007397133610725843
0.0076282940360610264
0.00785945446139621
0.008090614886731391
0.008321775312066574
0.008552935737401757
0.00878409616273694
0.009015256588072122
0.009246417013407305
0.009477577438742488
0.009708737864077669
0.009939898289412852
0.010171058714748035
0.010402219140083218
0.0106333795654184
0.010864539990753583
0.

In [20]:
recommended_books_df = random_recs_df
recommended_books_df.head()

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2,goodreads_book_id,isbn,isbn13,original_title,average_rating,ratings_count,book_authors,book_desc,book_isbn,book_title,genres,book_desc_tok
820,3495637,185,1392,4,19482,23,599.0,743236017,9.780743e+12,"Sex, Drugs, and Cocoa Puffs: A Low Culture Man...",3.75,56548.0,['Chuck Klosterman'],Countless writers and artists have spoken for ...,9.780740e+12,"Sex, Drugs, and Cocoa Puffs: A Low Culture Man...","[Nonfiction, Writing, Essays, Humor, Culture, ...",countless writers and artists ha...
3924,6472943,35464,9052,5,6784,478,16061295.0,1250045630,9.781250e+12,Sixth Grave on the Edge,4.38,17022.0,['Darynda Jones'],Few things in life can come between a grim rea...,9.781250e+12,Sixth Grave on the Edge,"[Fantasy, Paranormal, Fantasy, Urban Fantasy, ...",few things in life can come ...
2213,4643165,1254,6645,5,15144,371,138070.0,618194576,9.780618e+12,Flotsam,4.28,19541.0,['Stephanie Skeem'],Amy was cursed. There was no doubt about it. I...,9.781450e+12,Flotsam,"[Fantasy, Young Adult, Fiction, Childrens, Mid...",amy was cursed there was ...
1658,4304429,714,7830,3,241,70,41704.0,743449673,9.780743e+12,Shock Wave,3.93,12527.0,['Clive Cussler'],"From an island off the coast of Australia, a d...",9.780740e+12,Shock Wave,"[Adventure, Fiction, Thriller]",from an island off the coast ...
2061,4525472,808,7176,5,13752,462,93157.0,140436030,9.780140e+12,The Monk,3.80,13633.0,['Matthew Lewis'],Set in the sinister monastery of the Capuchins...,9.780140e+12,The Monk,"[Classics, Gothic, Horror, Fiction, Literature...",set in the sinister monastery ...


In [22]:
missing_orig_books_indices_list = []
for idx in list(set(recommended_books_df.index)):
    if idx not in orig_book_indices:
        missing_orig_books_indices_list.append(idx)

In [23]:
missing_orig_books_indices_list

[]

In [25]:
recommended_books_df.shape

(43260, 18)

In [26]:
#binarizer to transform between labels and a multilabel format
mlb = MultiLabelBinarizer()
#fit the binarizer to the list of genres
mlb.fit([unique_genres_list])

MultiLabelBinarizer()

## Step 1: 
Original books genres -> matrix

In [29]:
num_preds_per_book = 10
orig_book_genres_matrix = []
for orig_book_index in orig_book_indices:
    orig_book = books_df.iloc[books_df.index==orig_book_index]
    orig_book_genres = orig_book['genres']
    orig_book_genres_matrix.extend(np.repeat(mlb.transform(orig_book_genres), repeats = num_preds_per_book, axis=0))

In [30]:
y_expected = np.array(orig_book_genres_matrix)

In [31]:
np.array(y_expected).shape

(43260, 424)

## Step 2
Predictions -> matrix with size = number of genres

In [32]:
#Cleaning: if genre = NaN, replace with '[]'
recommended_books_df.loc[recommended_books_df['genres'].isna(), 'genres']='[]'

In [35]:
recommended_books_df['genres']

820     [Nonfiction, Writing, Essays, Humor, Culture, ...
3924    [Fantasy, Paranormal, Fantasy, Urban Fantasy, ...
2213    [Fantasy, Young Adult, Fiction, Childrens, Mid...
1658                       [Adventure, Fiction, Thriller]
2061    [Classics, Gothic, Horror, Fiction, Literature...
                              ...                        
2497              [Fiction, Classics, Literature, Novels]
1182    [Young Adult, Fantasy, Fantasy, Urban Fantasy,...
3480        [Young Adult, Romance, Contemporary, Fiction]
3768    [New Adult, Romance, Contemporary, Young Adult...
3918    [Historical, Historical Fiction, Fiction, Hist...
Name: genres, Length: 43260, dtype: object

In [36]:
#Cleaning
#recommended_books = recommended_books.dropna(subset=['genres'])
#recommended_books_df['genres'] = recommended_books_df['genres'].progress_apply(lambda x: ast.literal_eval(x))

In [37]:
recommended_books_genres = list(recommended_books_df['genres'])


In [38]:
recommended_books_genres

[['Nonfiction',
  'Writing',
  'Essays',
  'Humor',
  'Culture',
  'Pop Culture',
  'Music'],
 ['Fantasy',
  'Paranormal',
  'Fantasy',
  'Urban Fantasy',
  'Romance',
  'Fantasy',
  'Romance',
  'Paranormal Romance',
  'Mystery'],
 ['Fantasy',
  'Young Adult',
  'Fiction',
  'Childrens',
  'Middle Grade',
  'Realistic Fiction'],
 ['Adventure', 'Fiction', 'Thriller'],
 ['Classics', 'Gothic', 'Horror', 'Fiction', 'Literature', '18th Century'],
 ['Fiction', 'Young Adult', 'Childrens', 'Fantasy'],
 ['Romance',
  'Romance',
  'Historical Romance',
  'Historical',
  'Historical',
  'Historical Fiction',
  'Historical',
  'Regency'],
 ['Horror',
  'Fiction',
  'Science Fiction',
  'Fantasy',
  'Paranormal',
  'Vampires',
  'Apocalyptic',
  'Post Apocalyptic'],
 ['Historical',
  'Historical Fiction',
  'Fiction',
  'Cultural',
  'France',
  'Historical'],
 ['Fiction',
  'Childrens',
  'Young Adult',
  'Realistic Fiction',
  'Childrens',
  'Middle Grade'],
 ['Fiction'],
 ['History',
  'Nonfict

In [39]:
rec_books_matrix = mlb.transform(recommended_books_genres)

In [40]:
rec_books_matrix.shape

(43260, 424)

In [41]:
rec_books_matrix[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,

In [42]:
y_pred = rec_books_matrix
y_pred

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [44]:
print(classification_report(
    y_expected,
    y_pred, output_dict=False))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.00      0.00      0.00        40
           2       0.00      0.00      0.00       250
           3       0.00      0.00      0.00       110
           4       0.00      0.00      0.00        10
           5       0.00      0.00      0.00        20
           6       0.02      0.02      0.02       340
           7       0.01      0.01      0.01       120
           8       0.00      0.00      0.00        10
           9       0.01      0.01      0.01       580
          10       0.01      0.01      0.01       370
          11       0.00      0.00      0.00        20
          12       0.02      0.02      0.02      1040
          13       0.02      0.02      0.02       990
          14       0.04      0.04      0.04      1870
          15       0.00      0.00      0.00       300
          16       0.00      0.00      0.00       300
          17       0.00    